In [17]:
from sphedron import Icosphere, StratifiedIcospheres
from sphedron.utils import transform as sphetr
import numpy as np
from scipy.spatial import transform

In [18]:
# icos = Icosphere(depth=4)
icos =StratifiedIcospheres(factors=[1,2,2,2,2,4])
print(icos)

Mesh has:                
	 #vertices: 40962                
	 #faces: 88740,                
	 #edges: 266220,                
	 #edges_unique: 133110,                
	meta: {'depths': array([ 1,  2,  4,  8, 16, 64])}


In [19]:
def get_rotation_matrices_to_local_coordinates(
    reference_phi: np.ndarray,
    reference_theta: np.ndarray,
    rotate_latitude: bool,
    rotate_longitude: bool) -> np.ndarray:

  if rotate_longitude and rotate_latitude:

    # We first rotate around the z axis "minus the azimuthal angle", to get the
    # point with zero longitude
    azimuthal_rotation = - reference_phi

    # One then we will do a polar rotation (which can be done along the y
    # axis now that we are at longitude 0.), "minus the polar angle plus 2pi"
    # to get the point with zero latitude.
    polar_rotation = - reference_theta + np.pi/2

    return transform.Rotation.from_euler(
        "zy", np.stack([azimuthal_rotation, polar_rotation],
                       axis=1)).as_matrix()
  elif rotate_longitude:
    # Just like the previous case, but applying only the azimuthal rotation.
    azimuthal_rotation = - reference_phi
    return transform.Rotation.from_euler("z", -reference_phi).as_matrix()
  elif rotate_latitude:
    # Just like the first case, but after doing the polar rotation, undoing
    # the azimuthal rotation.
    azimuthal_rotation = - reference_phi
    polar_rotation = - reference_theta + np.pi/2

    return transform.Rotation.from_euler(
        "zyz", np.stack(
            [azimuthal_rotation, polar_rotation, -azimuthal_rotation]
            , axis=1)).as_matrix()
  else:
    raise ValueError(
        "At least one of longitude and latitude should be rotated.")

def latlong_to_phitheta(latlong: np.ndarray) -> np.ndarray:
  phi = np.deg2rad(latlong[:,[1]])
  theta = np.deg2rad(90 - latlong[:,[0]])
  return np.c_[phi,theta]
    
def spherical_to_cartesian(
    phi: np.ndarray, theta: np.ndarray
    ):
  # Assuming unit radius.
  return np.array([np.cos(phi)*np.sin(theta),
          np.sin(phi)*np.sin(theta),
          np.cos(theta)]).T

In [20]:
latlong = icos.vertices_latlong

In [21]:
phitheta = latlong_to_phitheta(latlong)

In [22]:
phitheta_sender = phitheta[icos.edges_unique[:,0]]
pos_sender = icos.vertices[icos.edges_unique[:,0]]
phitheta_receiver = phitheta[icos.edges_unique[:,1]]
pos_receiver = icos.vertices[icos.edges_unique[:,1]]

In [23]:
thetaphi = sphetr.latlong_to_thetaphi(icos.vertices_latlong)

In [24]:
np.linalg.norm(thetaphi[:,::-1]-phitheta)

np.float64(0.0)

In [25]:
def rotate_with_matrices(rotation_matrices: np.ndarray, positions: np.ndarray
                         ) -> np.ndarray:
  return np.einsum("bji,bi->bj", rotation_matrices, positions)

In [32]:
rot_mats = get_rotation_matrices_to_local_coordinates(reference_phi=phitheta_receiver[:,0], reference_theta=phitheta_receiver[:,1],
                                                      rotate_latitude=True,rotate_longitude=True)

In [33]:
pos = rotate_with_matrices(rot_mats, pos_sender)

In [34]:
# np.linalg.norm(pos-pos1)

In [35]:
pos

array([[ 4.47213595e-01,  1.16735757e-16,  8.94427191e-01],
       [ 4.47213595e-01,  2.76393202e-01,  8.50650808e-01],
       [ 4.47213595e-01, -2.76393202e-01,  8.50650808e-01],
       ...,
       [ 9.99793188e-01,  1.93422283e-02,  6.28162730e-03],
       [ 9.99850181e-01,  1.40009017e-02, -1.01779791e-02],
       [ 9.99850181e-01, -5.34070824e-03, -1.64649124e-02]],
      shape=(133110, 3))

In [36]:
rotate_with_matrices(rot_mats, pos_receiver)

array([[ 1.00000000e+00, -1.16735757e-16,  1.11022302e-16],
       [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  1.11022302e-16,  0.00000000e+00],
       ...,
       [ 1.00000000e+00,  0.00000000e+00, -1.00613962e-16],
       [ 1.00000000e+00,  0.00000000e+00, -3.46944695e-18],
       [ 1.00000000e+00,  0.00000000e+00, -3.46944695e-18]],
      shape=(133110, 3))

In [37]:
pos2 = sphetr.senders_xyz_in_receivers_rotation(receivers_thetaphi=phitheta_receiver[:,::-1], senders_xyz=pos_sender,
                                                zero_latitude=True, zero_longitude=True)

In [38]:
np.linalg.norm(pos-pos2)

np.float64(0.0)

In [40]:
phi = (1 + np.sqrt(5)) / 2
angle_between_faces = 2 * np.arcsin(phi / np.sqrt(3))
rotation_angle = (np.pi - angle_between_faces) / 2
rotation = transform.Rotation.from_euler(seq="y", angles=rotation_angle)
rotation_matrix = rotation.as_matrix()

In [41]:
rotation_matrix

array([[ 0.93417236,  0.        ,  0.35682209],
       [ 0.        ,  1.        ,  0.        ],
       [-0.35682209,  0.        ,  0.93417236]])

In [42]:
rotation_matrix.T

array([[ 0.93417236,  0.        , -0.35682209],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.35682209,  0.        ,  0.93417236]])